In [8]:
import pandas as pd
import numpy as np

coin_name = "KRW-XRP"
# df_org = pd.read_csv('./data/minute_index_result.csv') #예측모델
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
df_org.columns 

Index(['Unnamed: 0', 't', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99',
       'wma7', 'wma25', 'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s',
       'macd_d', 'rsi7', 'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh',
       'bhi', 'bl', 'bli', 'bm', 'bw', 'vwap'],
      dtype='object')

In [13]:
#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0 #누적 구매 수량
    buy_price = 0 #누적 구매 금액
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)


In [29]:
revenue_rate = 0.008  #익절 비율
max_loss_rate = 0.09   #손절 비율: 20%
increace_rate = 0.16  
buy_cnt_limit = 19     #최대 오픈 건수
buy_amt_unit = 33    #최소 오픈 수량

trade_fee = 0.001 #거래수수료
close = 400
buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)
buy_amt_list
print(df_org.shape)

(199670, 32)


In [30]:
buy_cnt = 0 
buy_price = 0 #누적 구매 금액
buy_amt = 0 #누적 구매 수량
revenue = 0 #전체 이익금
revenue_t = 0 #단계별 이익금
buy_cnt_tot = 0 
# df = df_org.iloc[df_org.shape[0]-144000:,]
df = df_org.iloc[df_org.shape[0]-144000:df_org.shape[0]-124000,]

df = df_org.iloc[df_org.shape[0]-144000:,] #1일 1440분, 144000분 -> 100일
for i in range(0, df.shape[0]-1):
    close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
    close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4)
    wma7 = round(df.iloc[i:i+1,]['wma7'].values[0],4) #7분 가중이동 평균
    wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4) #99분 가중이동평균
    vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4) # 거래량이동평균
    
    #stop loss
    loss = buy_price - close2*buy_amt
    if loss > max_loss:
        revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
        revenue = round(revenue + revenue_t,4)
        print("*[{}] stop loss:{} revenue:{} ".format(i, round(revenue_t,4), round(revenue,4)))        
        buy_cnt = 0
        buy_amt = 0
        buy_price = 0
        continue

    #take profit
    tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
    if buy_cnt > 0 and tp_revenue > 0:
        revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
        revenue = round(revenue + revenue_t,4)
#         print("*[{}] take profit:{} revenue:{} ".format(i, revenue_t, revenue))        
        buy_cnt = 0
        buy_amt = 0
        buy_price = 0
        continue
    
    #buy
    if buy_cnt < buy_cnt_limit and close2 < vwap and close2 < wma7 and wma7 > wma99:
        temp_amt = buy_amt_unit + buy_amt*increace_rate
        buy_price = round(buy_price + close2*temp_amt,4)
        buy_amt = round(buy_amt + temp_amt,4)
        buy_cnt = buy_cnt + 1 
    
#     print("c1:{} c2:{} vwap:{} olc:{} ola:{} olp:{} loss:{} tp_revenue:{} revenue:{}"
#       .format(close1, close2, vwap, buy_cnt, buy_amt, buy_price, loss, tp_revenue, revenue))
        
        
print(df.shape[0], revenue)

*[20619] stop loss:-124111.1628 revenue:9852.4283 
*[31259] stop loss:-119352.3565 revenue:-17684.7879 
*[60599] stop loss:-124746.2893 revenue:11798.5843 
*[64934] stop loss:-125326.0645 revenue:99407.0022 
*[68766] stop loss:-124796.3333 revenue:164640.4876 
*[71048] stop loss:-125211.5607 revenue:238869.3853 
*[72897] stop loss:-124983.0276 revenue:207609.817 
*[75231] stop loss:-122153.0288 revenue:250029.5944 
*[77329] stop loss:-121251.9669 revenue:141397.435 
*[77942] stop loss:-121883.0895 revenue:35264.9706 
*[84022] stop loss:-125889.1009 revenue:63543.1957 
*[96292] stop loss:-129997.6926 revenue:226472.2956 
*[99419] stop loss:-123175.363 revenue:122596.3101 
*[99923] stop loss:-119775.5536 revenue:2820.7565 
*[104609] stop loss:-128075.8609 revenue:-3446.4333 
*[108455] stop loss:-120939.1537 revenue:-119269.0998 
*[110610] stop loss:-122007.7665 revenue:-209169.0176 
*[125387] stop loss:-122122.4658 revenue:-223832.5821 
*[135317] stop loss:-119466.325 revenue:-167468.810

In [16]:
272015.8648

272015.8648